In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import requests
import pickle

# 配置web驱动器
driver = webdriver.Chrome()
# 设置等待时间
wait = WebDriverWait(driver, 10)

# 打开IMDb网站并执行搜索功能
def search_keyword(keyword):
    driver.get("https://www.imdb.com/")
    input_element = wait.until(EC.presence_of_element_located((By.ID, "suggestion-search")))
    input_element.send_keys(keyword + Keys.ENTER)

# 获得详情页和图片链接
def get_data():
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "ipc-metadata-list-summary-item")))
    xpath_expression = '//a[@class="ipc-metadata-list-summary-item__t"]'
    element = driver.find_element(By.XPATH, xpath_expression)
    element.click()

    soup = BeautifulSoup(driver.page_source, "html.parser")
    target_div = soup.find("div", class_="sc-4e4cc5f9-7 joCxEc")

    if target_div:
        img_element = target_div.find("img")
        title_element = target_div.find("a", class_="ipc-lockup-overlay ipc-focusable")
        
        if img_element and title_element:
            return driver.current_url, img_element["src"], title_element["aria-label"]

    print("未找到目标元素")
    return None, None, None

# 下载图片
def download_image(img_url, path):
    response = requests.get(img_url)
    if response.status_code == 200:
        with open(path, 'wb') as f:
            f.write(response.content)
        print("图片已成功下载：", path)
    else:
        print("下载失败，状态码：", response.status_code)

if __name__ == '__main__':
    dic = {}
    keyword_list = ['Jurassic Park (1993)', 'Star Trek III: The Search for Spock (1984)', 'Star Trek: The Motion Picture (1979)', "Bram Stoker's Dracula (1992)", 'Indiana Jones and the Last Crusade (1989)', 'Twister (1996)', 'Beauty and the Beast (1991)', 'Speed (1994)', 'Die Hard: With a Vengeance (1995)', 'Interview with the Vampire (1994)']  
    for keyword in keyword_list:
        search_keyword(keyword)
        url, img_url, title = get_data()

        if img_url and title:
            print("点击后的网址 URL:", url)
            print("图片链接:", img_url)
            print("电影名称及演员列表:", title)
            # 文件名包含字符串，数字容易匹配不到
            new_word = ''.join(char for char in keyword if char.isalpha())
            pic_dir = f"pictures/{new_word}.jpg"
            download_image(img_url, "static/"+pic_dir)
#             update_file(keyword, [url, pic_dir])
            dic[keyword] =  [url, pic_dir]
        

import os
import pickle

pickle_name = "movie_url_pic.pickle"
# 检查本地是否存在 movie_url_pic.pickle 文件
if os.path.exists(pickle_name):
    # 如果存在，则读取该文件并加载其内容为字典
    with open(pickle_name, 'rb') as f:
        movie_url_pic_dict = pickle.load(f)
else:
    # 如果不存在，则创建一个空字典
    movie_url_pic_dict = {}

# 将两个字典融合
merged_dict = {**movie_url_pic_dict, **dic}

# 输出融合后的字典
print(merged_dict)
with open(pickle_name, 'wb') as f:
    pickle.dump(merged_dict, f)

The chromedriver version (122.0.6261.128) detected in PATH at /opt/homebrew/bin/chromedriver might not be compatible with the detected chrome version (123.0.6312.122); currently, chromedriver 123.0.6312.122 is recommended for chrome 123.*, so it is advised to delete the driver in PATH and retry


点击后的网址 URL: https://www.imdb.com/title/tt0107290/?ref_=fn_al_tt_1
图片链接: https://m.media-amazon.com/images/M/MV5BMjM2MDgxMDg0Nl5BMl5BanBnXkFtZTgwNTM2OTM5NDE@._V1_QL75_UX190_CR0,2,190,281_.jpg
电影名称及演员列表: View ’Jurassic Park’ Poster
图片已成功下载： static/pictures/JurassicPark.jpg
点击后的网址 URL: https://www.imdb.com/title/tt0088170/?ref_=fn_al_tt_1
图片链接: https://m.media-amazon.com/images/M/MV5BMmFiMTczYjgtMWQwOC00MjFlLWFlZjUtMjJjNTcxNjAzY2Q1XkEyXkFqcGdeQXVyMjUzOTY1NTc@._V1_QL75_UX190_CR0,0,190,281_.jpg
电影名称及演员列表: View ’Star Trek III: The Search for Spock’ Poster
图片已成功下载： static/pictures/StarTrekIIITheSearchforSpock.jpg
点击后的网址 URL: https://www.imdb.com/title/tt0079945/?ref_=fn_al_tt_1
图片链接: https://m.media-amazon.com/images/M/MV5BNjk1ZjAyZjktZTY4YS00NDY3LWIwMzktMjZiNGIzODFiZDVmXkEyXkFqcGdeQXVyMjUzOTY1NTc@._V1_QL75_UX190_CR0,2,190,281_.jpg
电影名称及演员列表: View ’Star Trek: The Motion Picture’ Poster
图片已成功下载： static/pictures/StarTrekTheMotionPicture.jpg
点击后的网址 URL: https://www.imdb.com/title/tt0103874/?ref_

In [33]:
with open(pickle_name, 'rb') as f:
    movie_url_pic_dict = pickle.load(f)

In [35]:
movie_url_pic_dict['Jurassic Park (1993)']

['https://www.imdb.com/title/tt0107290/?ref_=fn_al_tt_1',
 './pictures/Jurassic Park (1993).jpg']

In [43]:
keyword

'Interview with the Vampire (1994)'

In [44]:
clean_title = ''.join(char for char in keyword if char.isalpha())
clean_title

'InterviewwiththeVampire'

In [42]:
for key in movie_url_pic_dict:
    new_key = ''.join(char for char in key if char.isalpha()) 
    print(new_key)
    movie_url_pic_dict[]

UleesGold
ChasingAmy
SevenYearsinTibet
Amistad
BoogieNights
EveryoneSaysILoveYou
JerryMaguire
MrsBrownHerMajestyMrsBrown
Emma
Mother
SaintThe
EnglishPatientThe
PeacemakerThe
Titanic
InOut
IKnowWhatYouDidLastSummer
Evita
Volcano
Gattaca
JurassicPark
StarTrekIIITheSearchforSpock
StarTrekTheMotionPicture
BramStokersDracula
IndianaJonesandtheLastCrusade
Twister
BeautyandtheBeast
Speed
DieHardWithaVengeance
InterviewwiththeVampire
